In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np # linear algebra
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import plotly.express as px


import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from statsmodels.tsa.seasonal import seasonal_decompose

from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

In [0]:
test= pd.read_csv('/content/drive/My Drive/dacon/이상치처리한df_for_lgbm.csv')
w_hour = pd.read_excel('/content/drive/My Drive/dacon/인천_시간별__기상자료(16-18)_축소__7월1일.xlsx')

In [0]:
test2 = test.copy()
test2['Time'] = pd.to_datetime(test2['Time'])
test2 = test2.set_index('Time')
place_id=[]; time=[] ; target=[] # 빈 리스트를 생성합니다.
for i in test2.columns:
    for j in range(len(test2)):
        place_id.append(i) # place_id에 미터 ID를 정리합니다.
        time.append(test2.index[j]) # time에 시간대를 정리합니다.
        target.append(test2[i].iloc[j]) # target에 전력량을 정리합니다.

new_df=pd.DataFrame({'place_id':place_id,'time':time,'target':target})
new_df=new_df.dropna() # 결측치를 제거합니다.
new_df=new_df.set_index('time') # time을 인덱스로 저장합니다.
new_df.head()

,place_id,target
time,,
2017-07-01 00:00:00,X125,0.226
2017-07-01 01:00:00,X125,0.223
2017-07-01 02:00:00,X125,0.216
2017-07-01 03:00:00,X125,0.224
2017-07-01 04:00:00,X125,0.303


## w_hour 전처리

In [0]:
#test = new_df.copy()

In [0]:
w_hour.tail()

,지점,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),날씨,전운량(10분위)
17083,112,2018.7.1 19:00,21.6,1.8,5.1,98.0,NaN,약한비계속,10.0
17084,112,2018.7.1 20:00,21.4,1.3,5.0,98.0,NaN,약한비계속,10.0
17085,112,2018.7.1 21:00,21.3,4.2,4.5,98.0,NaN,보통비계속,10.0
17086,112,2018.7.1 22:00,21.3,6.2,4.8,98.0,NaN,보통비계속,10.0
17087,112,2018.7.1 23:00,21.2,4.6,4.1,98.0,NaN,보통비계속,10.0


In [0]:
w_hour['강수량(mm)'] = w_hour['강수량(mm)'].fillna(0) # 비 오는 날은 결측치 없음
w_hour = w_hour.rename(columns={'일시':'time'})
w_hour['time'] = pd.to_datetime(w_hour['time']) 
w_hour =w_hour.drop('지점', axis=1)

In [0]:
w_hour = w_hour.interpolate(method='linear', limit_direction='both')

In [0]:
w_hour.isna().sum()

time            0
기온(°C)          0
강수량(mm)         0
풍속(m/s)         0
습도(%)           0
적설(cm)          0
날씨           3767
전운량(10분위)       0
dtype: int64

In [0]:
new_df = pd.merge(new_df.reset_index(), w_hour, on = 'time')

## w_day 전처리

In [0]:
w_day = pd.read_excel('/content/drive/My Drive/dacon/인천_일별_기상자료(16-18)_축소.xlsx')
w_day_selected  = w_day[['일시', '평균기온(°C)', '최저기온(°C)','최고기온(°C)' ]]
w_day_selected = w_day_selected.rename(columns={'일시':'time'})
w_day_selected['time'] = pd.to_datetime(w_day_selected['time']) 
w_day_selected['temp_ma7'] = w_day_selected['평균기온(°C)'].rolling(window=7).mean()
w_day_selected['date'] = w_day_selected['time'].dt.date
del w_day_selected['time']

In [0]:
w_hour.columns

Index(['time', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '적설(cm)', '날씨',
       '전운량(10분위)'],
      dtype='object')

In [0]:
new_df = new_df.reset_index()
new_df['date'] = new_df['time'].dt.date

In [0]:
new_df=pd.merge(new_df, w_day_selected, on = 'date', how='left')
del new_df['date']

## 시간 관련 변수 추가

In [0]:
new_df['Day of Week'] = new_df['time'].dt.weekday_name
new_df['hour'] = new_df['time'].dt.hour
new_df['month'] = new_df['time'].dt.month

dict_day = {'Sunday': 0, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4,
       'Friday':5,'Saturday':6}

new_df['Day of Week'] = new_df['Day of Week'].map(dict_day)

In [0]:
# 요일별 시간 당 수요 평균 변수
hour_mean = new_df.groupby(['place_id','Day of Week','hour']).mean()[['target']].reset_index()
hour_mean = hour_mean.rename(columns={'target':'hour mean per day'})
new_df = pd.merge(new_df, hour_mean, on=['place_id','Day of Week','hour'])

In [0]:
new_df.columns

Index(['index', 'time', 'place_id', 'target', '기온(°C)', '강수량(mm)', '풍속(m/s)',
       '습도(%)', '적설(cm)', '날씨', '전운량(10분위)', '평균기온(°C)', '최저기온(°C)',
       '최고기온(°C)', 'temp_ma7', 'Day of Week', 'hour', 'month',
       'hour mean per day'],
      dtype='object')

In [0]:
new_df2 = new_df[['time', 'place_id', 'target','hour','Day of Week', 'month','기온(°C)', '강수량(mm)', '풍속(m/s)',
       '습도(%)','전운량(10분위)','최저기온(°C)','최고기온(°C)','temp_ma7','hour mean per day']]

## 결측치 df 만들기

In [0]:
test_w = test2.reset_index()

In [0]:
test_w = test_w.rename(columns={'Time':'time'})

In [0]:
# 시간 기상 
test_w = pd.merge(test_w, w_hour, on = 'time')
# 일 기상
test_w['date'] = test_w['time'].dt.date
test_w=pd.merge(test_w, w_day_selected, on = 'date', how='left')
del test_w['date']

In [0]:
# 시간 관련
test_w['Day of Week'] = test_w['time'].dt.weekday_name
test_w['hour'] = test_w['time'].dt.hour
test_w['month'] = test_w['time'].dt.month

dict_day = {'Sunday': 0, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4,
       'Friday':5,'Saturday':6}

test_w['Day of Week'] = test_w['Day of Week'].map(dict_day)

In [0]:
new_df2.columns

Index(['time', 'place_id', 'target', 'hour', 'Day of Week', 'month', '기온(°C)',
       '강수량(mm)', '풍속(m/s)', '습도(%)', '전운량(10분위)', '최저기온(°C)', '최고기온(°C)',
       'temp_ma7', 'hour mean per day'],
      dtype='object')

In [0]:
test_w.columns[200:]

Index(['X481', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '적설(cm)', '날씨',
       '전운량(10분위)', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', 'temp_ma7',
       'Day of Week', 'hour', 'month'],
      dtype='object')

In [0]:
test_w = test_w.drop(['적설(cm)', '날씨','평균기온(°C)'],axis=1)

## 베이지안 opt

In [0]:
!pip install Bayesian-Optimization

  Created wheel for Bayesian-Optimization: filename=bayesian_optimization-1.0.1-cp36-none-any.whl size=10032 sha256=8392f2477083f00cc5a7f6d408d0742c97e2f29f42594abc0c1a8bbc029b95ad
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built Bayesian-Optimization


In [0]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold


def LGB_bayesian(
    num_leaves, 
    bagging_fraction,
    feature_fraction,
    min_child_weight, 
    min_data_in_leaf,
    min_split_gain,
    max_depth,
    lambda_l1,
    lambda_l2
 #   learning_rate
 #   num_iterations,
    #subsample_freq
     ):
    
    # LightGBM expects next three parameters need to be integer. 
    num_leaves = int(num_leaves)
    min_data_in_leaf = int(min_data_in_leaf)
    max_depth = int(max_depth)

    assert type(num_leaves) == int
    assert type(min_data_in_leaf) == int
    assert type(max_depth) == int
    

    param = {
              'num_leaves': int(round(num_leaves)), 
             'min_data_in_leaf': int(round(min_data_in_leaf)),
              'min_child_weight': min_child_weight,
              'feature_fraction' : feature_fraction,
              'bagging_fraction' : bagging_fraction,
           #   'learning_rate' : learning_rate,
              'learning_rate' : 0.05,
              'max_depth': int(round(max_depth)),
              'lambda_l1': lambda_l1,
              'lambda_l2': lambda_l2,
              #'subsample_freq ': subsample_freq,
              'n_estimators' : 1000,
              'objective': 'regression',
              'min_split_gain': min_split_gain,
              'save_binary': True,
              'n_estimators': 10000,
              'early_stopping_round': 300,
              'seed': 42,
              'feature_fraction_seed': 42,
              'drop_seed': 42,
              'data_random_seed': 42,
              'boosting_type': 'gbdt',
              'verbose': 1,
              'boost_from_average': True,
              'metric':{'rmse'}}  
    folds = KFold(n_splits=5, random_state=42)
    folds_generator = folds.split(X1, y1.values)
    
    
    
    cv_result = lgb.cv(param, train_data, folds=folds_generator, seed= 42, stratified=False, verbose_eval =5000, 
                       metrics=['rmse']) # stratified=False가 회귀, True 는 분류
    return min(cv_result['rmse-mean'])*(-1)

In [0]:
# Bounded region of parameter space
bounds_LGB = {
    'num_leaves': (5, 50), # 크면 오버핏
    'min_data_in_leaf': (100, 500), # 작으면 오버핏
    'bagging_fraction' : (0.0001, 0.6),
    'feature_fraction' : (0.0001, 0.6),
 #   'learning_rate': 0.01,
    'min_child_weight': (0.0001, 0.01), # 크면 오버핏
    'min_split_gain': (0.001, 0.1),
    'lambda_l1': (0.00001, 5), 
    'lambda_l2': (0.00001, 3),
    'max_depth':(5,12),
    
    #'subsample_freq ': (0.1, 0.9),
   # 'n_estimator' : (10000.20000),

}

## 예측

In [0]:
#test_second = test_w.copy()

In [0]:
# 아래 식이 잘못될 경우
test_w = test_second.copy()

In [0]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from time import time

rmse_std = []
rmse_sum = []
folds = KFold(n_splits=5)
fig = plt.figure(figsize=(20,10))
for meter in test_w.columns[1:201].unique():
  training_start_time = time()
  Xx = new_df2[new_df2['place_id']==meter]
  X11, X22 = train_test_split(Xx, test_size=0.2, random_state=42)
  X11 = X11.reset_index().drop('index', axis=1)
  X22 = X22.reset_index().drop('index', axis=1)

  X1 = X11.drop(['time', 'place_id', 'target'],axis=1)
  y1= X11[X11.columns[2]]

  X2 = X22.drop(['time', 'place_id', 'target'],axis=1)
  y2= X22[X22.columns[2]]

  #X = Xx.drop(['time', 'place_id', 'target'],axis=1).reset_index().drop('index', axis=1)
  y = Xx[Xx.columns[2]].reset_index().drop('index', axis=1)


  # 베이지안 opt
  LGB_BO = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=42)
  train_data = lgb.Dataset(data=X1, label=y1, free_raw_data=False)

  init_points = 10
  n_iter = 8

  print('-' * 130)

  with warnings.catch_warnings():
      warnings.filterwarnings('ignore')
      LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.01, alpha=1e-6)
  
  params = LGB_BO.max['params']
  params['num_leaves'] = int(round(params['num_leaves'])) 
  params['min_data_in_leaf'] = int(round(params['min_data_in_leaf']))
  params['max_depth'] = int(round(params['max_depth']))
  params['n_estimators ']= 10000,
  params['learning_rate ']= 0.01,
  params['objective'] = 'regression'
  params['metric'] = 'rmse'
  params['verbosity'] = -1
  params['boosting_type'] = 'gbdt'
  params['random_state'] = 47

  # 최적의 파라미터로 5-fold validation
  folds = KFold(n_splits=5)
  rmses = list()
  feature_importances = pd.DataFrame()
  feature_importances['feature'] = X1.columns

  
  for fold, (trn_idx, test_idx) in enumerate(folds.split(X1, y1)):
      start_time = time()
      print('Training on fold {}'.format(fold + 1))
      
      trn_data = lgb.Dataset(X1.iloc[trn_idx], label=y1.iloc[trn_idx])
      val_data = lgb.Dataset(X1.iloc[test_idx], label=y1.iloc[test_idx])
      clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds=500)
      
      feature_importances['fold_{}'.format(fold + 1)] = clf.feature_importance()
      rmses.append(clf.best_score['valid_1']['rmse'])
      
      pred += clf.predict(X2) / 5
      print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
  print('-' * 30)
  print(meter)
  print('Training has finished.')
  print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
  print('Mean rmse:', np.mean(rmses))
  print('-' * 30)

  # test!!
  
  rmse = sqrt(mean_squared_error(y2, pred))
  print('rmse={0}, std={1}, mean={2}, rmse/std={3}'.format(rmse, np.mean(y['target']), np.std(y['target']), (rmse/np.std(y['target']))))
  rmse_std.append((rmse/np.std(y['target'])))
  rmse_sum.append(rmse)
  # 결측치 예측
  ex = test_w[test_w[meter].isna() | (test_w[meter] == -9)][['time',meter,'hour', 'Day of Week', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)',
       '전운량(10분위)', '최저기온(°C)', '최고기온(°C)', 'temp_ma7']] # 결측인 행만 추출
  ex[meter] = clf.predict(ex[['hour', 'Day of Week', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)',
       '전운량(10분위)', '최저기온(°C)', '최고기온(°C)', 'temp_ma7']]) # 
  ex = ex[['time',meter]]
  ex.columns = ['time','fillna']

  test3 = pd.merge(test_w, ex, on ='time', how='left') # test3 재정의(병합)
  test3[meter] = np.where((test3[meter].isna()) | (test3[meter] == -9), test3['fillna'], test3[meter])
  test_w[meter] = test3[meter]


  print('num of null: ',meter, ' : ',test_w[meter].isna().sum())

  # 그래프 그리기
  #fig = plt.figure(figsize=(20,10))
  #ax2 = fig.add_subplot(2, 2, 1)
  #ax1 = fig.add_subplot(2, 2, 2)
  #ax2.plot(test3.set_index('Time')[meter])
  #ax2.plot(test_second.set_index('Time')[meter], color='r')

  #ax1.plot(test3.set_index('Time')[meter].resample('M').sum())
  #ax1.twinx().plot(test3.set_index('Time')['기온(°C)'].resample('M').mean(), color='r')
  #plt.title(meter)
  #plt.show()

  print(meter)

----------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        | -0.6395   |  0.2248   |  0.5704   |  3.66     |  1.796    |  6.092    |  0.001644 |  123.2    |  0.08675  |  32.05    |
[5000]	cv_agg's rmse: 0.878809 + 0.0335676
|  2        | -0.8775   |  0.4249   |  0.01245  |  4.85     |  2.497    |  6.486    |  0.0019   |  173.4    |  0.03112  |  28.61    |
|  3        | -0.6724   |  0.2592   |  0.1748   |  3.059    |  0.4185   |  7.045    |  0.003727 |  282.4    |  0.07873  |  13.99    |
|  4        | -0.6629   |  0.3086   |  0.3555   |  0.2323   |  1.823    |  6.194    |  0.000744 |  479.6    |  0.0966   |  41.38    |
[5000]	cv_agg's rmse: 

<Figure size 1440x720 with 0 Axes>

In [0]:
print(np.mean(rmse_sum), np.mean(rmse_std))

0.2259880359469858 0.7090840259015596


In [0]:
test3.to_csv('/content/drive/My Drive/dacon/test_lgbm_interpol_1119.csv',index=False)